# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
#print(len(full_data_rows_list))
# to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Making the connection 
from cassandra.cluster import Cluster
cluster = Cluster()

# Creating a session to establish connection and to execute queries in further steps
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace for Data Modeling with Cassandra project
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS <keyname>
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting up the KEYSPACE 
try:
    session.set_keyspace('<keyname>')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## requirment is to fetch the artist, song title and song's length in the music app history for sessionId = 338, and itemInSession = 4

# for this requirment we are creating a music app history table with the primary key as session id and clustering column as itemInSession to uniquely identify each row, 
# so that we can access all unique records with these two combinations.

#Creating the table for 1st requirement
q1_0= "create table if not exists music_app_history"
q1_0 = q1_0+ "(session_id int, iteminsession int, artist text, song_title text, song_length text, Primary Key (session_id, iteminsession) )"

try:
    session.execute(q1_0)
except Exception as e:
    print(e)

In [9]:
# Reading the data from the source file below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # this command will skip file header
    for line in csvreader:
## insert statement to load data into the music app history table
        q1_1 = "insert into music_app_history (session_id, iteminsession, artist, song_title, song_length ) "
        q1_1 = q1_1 + "values (%s,%s,%s,%s,%s)"
        ## assigning the appropriate column values from the file for each table column in the INSERT statement.
        session.execute(q1_1, (int(line[8]), int(line[3]), line[0], line[9], line[5])) 
        

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Writing the select statment to fetch the required data as asked in the first requirement.
query_1 = "select artist, song_title, song_length from music_app_history where session_Id = 338 and iteminsession = 4 "

try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)


Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## requirment-2 is to fetch name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

# for this requirment we are creating a music app history user table with the primary key as combinition of (user_id, session_id)and clustering column as itemInSession to uniquely identify each row, 
# so that we can access all unique records with these combinations.

#Creating the table for given requirement
q2_0= "create table if not exists music_app_history_user"
q2_0 = q2_0+ "(session_id int, user_id int, iteminsession int, user_first_name text, user_last_name text, artist_name text, song_title text,  Primary Key ((user_id, session_id), iteminsession) )"

try:
    session.execute(q2_0)
except Exception as e:
    print(e)
                    

In [12]:
# Reading the data from the source file below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    for line in csvreader:
## ## insert statement to load data into the music app history user table
        q2_1 = "insert into music_app_history_user (user_id, session_id, iteminsession, artist_name, song_title, user_first_name, user_last_name) "
        q2_1 = q2_1 + "values (%s,%s,%s,%s,%s,%s,%s)"
        ## assigning the appropriate column values from the file for each table column in the INSERT statement.
        session.execute(q2_1, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
   

In [13]:
## Writing the select statment to fetch the required data as asked in the 2nd requirement.
query_2 = "select itemInSession, artist_name, song_title, user_first_name, user_last_name from music_app_history_user where session_Id = 182 and user_id = 10 "

try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title , row.user_first_name+' '+row.user_last_name)
#     print (row.iteminsession, row.artist_name, row.song_title , row.user_first_name, row.user_last_name) #for checking the sorted values


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## requirment-3 is to fetch every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# for this requirment we are creating a my music app history table with the primary key song_title and clustering column as user_id to uniquely identify each row, 
# so that we can access all unique records with these combinations.

#Creating the table for given requirement

q3_0= "create table if not exists my_music_app_history"
q3_0 = q3_0+ "( song_title text, user_id int, user_first_name text, user_last_name text, Primary Key (song_title, user_id) )"

try:
    session.execute(q3_0)
except Exception as e:
    print(e)
                

                    

In [16]:
# Reading the data from the source file below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # to skip header
    for line in csvreader:
## ## insert statement to load data into the music app history user table
        q3_1 = "insert into my_music_app_history (song_title, user_id, user_first_name, user_last_name) "
        q3_1 = q3_1 + "values (%s,%s,%s,%s)"
        ## assigning the appropriate column values from the file for each table column in the INSERT statement.
        session.execute(q3_1, ( line[9], int(line[10]), line[1], line[4]))
   

In [17]:
## Writing the select statment to fetch the required data as asked in the 3rd requirement.

query_3 = "select song_title, user_id, user_first_name, user_last_name from my_music_app_history where song_title='All Hands Against His Own' "

try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title , row.user_first_name+' '+row.user_last_name)


All Hands Against His Own Jacqueline Lynch
All Hands Against His Own Tegan Levine
All Hands Against His Own Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
#Dropping all the tables prior to session closure

try:
    session.execute("drop table music_app_history ")
except Exception as e:
    print(e)

try:
    session.execute("drop table music_app_history_user ")
except Exception as e:
    print(e)

try:
    session.execute("drop table my_music_app_history ")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()